<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment03/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING DATASET AND INSTALLING REQUIRED DEPENDENCIES

In [24]:
#Importing dataset from drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
#getting the dataset
!unzip '/content/drive/MyDrive/lexicons.zip'

Archive:  /content/drive/MyDrive/lexicons.zip
replace lexicons/ta.translit.sampled.test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [26]:
!pip install wandb

In [27]:
import wandb
wandb.login()

wandb: Currently logged in as: kodikarthik21 (use `wandb login --relogin` to force relogin)


True

# IMPORTING NECESSARY LIBRARIES

In [28]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from wandb.keras import WandbCallback
from math import log

# DATASET PREPROCESSING

In [29]:
#converting tsv file to pandas dataframe for easier processing
train_df = pd.read_csv("/content/lexicons/ta.translit.sampled.train.tsv", sep = '\t', header = None)
train_df = train_df.dropna(axis=0)

val_df = pd.read_csv("/content/lexicons/ta.translit.sampled.dev.tsv", sep = '\t', header = None)
val_df = val_df.dropna(axis=0)

test_df = pd.read_csv("/content/lexicons/ta.translit.sampled.test.tsv", sep = '\t', header = None)
test_df = test_df.dropna(axis=0)

In [30]:
train_df

,0,1,2
0,ஃபியட்,fiat,2
1,ஃபியட்,phiyat,1
2,ஃபியட்,piyat,1
3,ஃபிரான்ஸ்,firaans,1
4,ஃபிரான்ஸ்,france,2
...,...,...,...
68213,ஹோல்ட்,holtt,1
68214,ஹோல்ட்,hoold,1
68215,ஹோல்ட்,hoolt,1
68216,ஹோல்ட்,hooltt,1


## Train dataset

In [31]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for input_text,target_text in zip(train_df[1][0:],train_df[0][0:]):
    target_text = "\t" + target_text + "\n"  #appending start and end tokens
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

#getting the list of input characters and making a dictionary for ease of processing
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
pred_target_token_index = dict([(i, char) for i, char in enumerate(target_characters)])

#adding the space token to both input and target dictionaries
if ' ' not in input_token_index:
  input_token_index[' '] = len(input_token_index)

if ' ' not in target_token_index:
  target_token_index[' '] = len(target_token_index)

pred_target_token_index[len(pred_target_token_index)] = ' '


num_encoder_tokens = len(input_characters) + 1
num_decoder_tokens = len(target_characters) + 1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

#getting the matrix representation of input and target data using the dictionaries defined earlier
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[char]
    encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :] = target_token_index[" "]
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

Number of samples: 68215
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [32]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 48,
 'ஃ': 2,
 'அ': 3,
 'ஆ': 4,
 'இ': 5,
 'ஈ': 6,
 'உ': 7,
 'ஊ': 8,
 'எ': 9,
 'ஏ': 10,
 'ஐ': 11,
 'ஒ': 12,
 'ஓ': 13,
 'க': 14,
 'ங': 15,
 'ச': 16,
 'ஜ': 17,
 'ஞ': 18,
 'ட': 19,
 'ண': 20,
 'த': 21,
 'ந': 22,
 'ன': 23,
 'ப': 24,
 'ம': 25,
 'ய': 26,
 'ர': 27,
 'ற': 28,
 'ல': 29,
 'ள': 30,
 'ழ': 31,
 'வ': 32,
 'ஷ': 33,
 'ஸ': 34,
 'ஹ': 35,
 'ா': 36,
 'ி': 37,
 'ீ': 38,
 'ு': 39,
 'ூ': 40,
 'ெ': 41,
 'ே': 42,
 'ை': 43,
 'ொ': 44,
 'ோ': 45,
 'ௌ': 46,
 '்': 47}

## Validation dataset

In [33]:
# Vectorize the data.
val_input_texts = []
val_target_texts = []

for val_input_text,val_target_text in zip(val_df[1][0:],val_df[0][0:]):
    val_target_text = "\t" + val_target_text + "\n"
    val_input_texts.append(val_input_text)
    val_target_texts.append(val_target_text)

val_encoder_input_data = np.zeros(
    (len(val_input_texts), max_encoder_seq_length), dtype="float32"
)
val_decoder_input_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length), dtype="float32"
)
val_decoder_target_data = np.zeros(
    (len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t] = input_token_index[char]
    val_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

## Test dataset

In [34]:
# Vectorize the data.
test_input_texts = []
test_target_texts = []

for test_input_text,test_target_text in zip(test_df[1][0:],test_df[0][0:]):
    test_target_text = "\t" + test_target_text + "\n"
    test_input_texts.append(test_input_text)
    test_target_texts.append(test_target_text)

test_encoder_input_data = np.zeros(
    (len(test_input_texts), max_encoder_seq_length), dtype="float32"
)
test_decoder_input_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length), dtype="float32"
)
test_decoder_target_data = np.zeros(
    (len(test_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1 :] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1 :] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# SWEEP

## Defining model

In [35]:
def rnnmodel(cell_name, embedding_size, hidden_size, num_enc_layers, num_dec_layers, dropout, r_dropout, batch_size, num_epochs, optimizer):
  
  cell_type = { "RNN": keras.layers.SimpleRNN,
                "GRU": keras.layers.GRU,
                "LSTM": keras.layers.LSTM
  }

  learning_rate = 0.01
  decay_rate = 0

  optimizers = {"adam": tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate),
                "nadam": tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate),
                "rmsprop": tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate),
                "adagrad": tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
  }

  # Define an input sequence and process it.
  encoder_inputs = keras.layers.Input(shape=(None,), name="input_1")
  embedding = keras.layers.Embedding(num_encoder_tokens, embedding_size, name="embedding_1")(encoder_inputs)
  encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_1")(embedding)
  for i in range(1, num_enc_layers):
    encoder_seq, *encoder_state = cell_type[cell_name](hidden_size, return_sequences=True,return_state = True, name="encoder_"+str(i+1), dropout=dropout,
                                                       recurrent_dropout=r_dropout)(encoder_seq)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.layers.Input(shape=(None,), name="input_2")
  decoder_embedding = keras.layers.Embedding(num_decoder_tokens, embedding_size, name="embedding_2")(decoder_inputs)

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_seq, *_ = cell_type[cell_name](hidden_size,return_sequences=True, return_state=True, name="decoder_1")(decoder_embedding,initial_state=encoder_state)
  for i in range(1, num_dec_layers):
    decoder_seq, *_ = cell_type[cell_name](hidden_size, return_sequences=True, return_state=True, name="decoder_"+str(i+1),dropout=dropout,
                                      recurrent_dropout=r_dropout)(decoder_seq,initial_state=encoder_state)

  decoder_outputs = keras.layers.TimeDistributed(keras.layers.Dense(num_decoder_tokens, activation="softmax"),name = 'time_distributed')(decoder_seq)
  #decoder_outputs = keras.layers.Dense(num_decoder_tokens, activation="softmax", name="dense_1")(decoder_seq)


  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(
    optimizer=optimizers[optimizer], loss="categorical_crossentropy", metrics=["accuracy"]
  )
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=batch_size,
      epochs=num_epochs,
      validation_data=([val_encoder_input_data, val_decoder_input_data],
                      val_decoder_target_data),
      callbacks=[WandbCallback()]
  )

  return model

## Helper functions for sweep

In [36]:
#----------------DEFINING FUNCTION FOR INFERENCE--------------------------------
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    #print(input_seq)
    states_value = encoder_model.predict([input_seq])
    #print(len(states_value))
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        #print([target_seq] + states_value)
        output_tokens, *decoder_states = decoder_model.predict([target_seq, states_value])
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = decoder_states
        #print(len(decoded_sentence))
    decoded_sentence = decoded_sentence.strip('\n')
    decoded_sentence = decoded_sentence.strip(' ')
    return decoded_sentence

#-------CALCULATING WORD-LEVEL TEST ACCURACY BY SAMPLE-BY-SAMPLE INFERENCE------
def calc_test_acc(model,config):

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  latent_dim = config.hidden_size
  encoder_inputs = model.input[0]  # input_1
  encoder_seq,*encoder_states  = model.get_layer('encoder_{}'.format(config.num_enc_layers)).output
  #encoder_seq,state_h_enc, state_c_enc = encoder_lstm2(encoder_seq)  # lstm_
  encoder_model = keras.Model(encoder_inputs, encoder_states)


  decoder_inputs = keras.layers.Input(shape=(1,), name="input_2") # input_2
  decoder_embed = model.get_layer('embedding_2')
  decoder_input = decoder_embed(decoder_inputs)

  if(config.cell_name == 'LSTM'):
    decoder_state_input_h = keras.layers.Input(shape=(latent_dim,), name="input_3")
    decoder_state_input_c = keras.layers.Input(shape=(latent_dim,), name="input_4")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  else:
    decoder_states_inputs = keras.layers.Input(shape=(latent_dim,), name="input_3")

  decoder_lstm1 = model.get_layer('decoder_1')
  decoder_outputs, *decoder_states = decoder_lstm1(
      decoder_input, initial_state=decoder_states_inputs
  )

  for i in range(config.num_dec_layers-1):
    decoder_outputs, *decoder_states = model.get_layer('decoder_'+str(i+2))(
        decoder_outputs,initial_state=decoder_states_inputs
    )

  decoder_dense = model.get_layer('time_distributed')
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs, decoder_states_inputs], [decoder_outputs] + decoder_states
  )

  f = open("predictions_vanilla1.txt", "a")
  g = open("correct_predictions_vanilla1.txt", "a")
  pred = []
  count = 0
  for i in range(test_df.shape[0]):
    decoded_sentence = decode_sequence(test_encoder_input_data[i:i+1],encoder_model,decoder_model)
    pred.append(decoded_sentence)
    f.write("Input:" + test_df[1][i] + " True output:" + test_df[0][i] + " Predicted output:" + decoded_sentence + '\n')
    print(i,pred[i])
    if(pred[i] == test_df[0][i]):
      count = count +1
      g.write("Input:" + test_df[1][i] + " True output:" + test_df[0][i] + " Predicted output:" + decoded_sentence + '\n')
  f.close()
  g.close()
  true_test_acc = count/test_df.shape[0]

  return true_test_acc

In [37]:
#--------------WORD-LEVEL VALIDATION ACCURACY FOR SWEEPING----------------------
def word_level_acc(model):
  val_results = model.predict([val_encoder_input_data,val_decoder_input_data])
  val_pred = {}
  for i in range(val_results.shape[0]):
    for j in range(max_decoder_seq_length):
      if(j==0):
        val_pred[i] = pred_target_token_index[np.argmax(val_results[i],axis = 1)[j]]
      else:
        val_pred[i] = val_pred[i] + pred_target_token_index[np.argmax(val_results[i],axis = 1)[j]]
      #print(pred_target_token_index[np.argmax(val_results[i],axis = 1)[j]])
    val_pred[i] = val_pred[i].replace('\n', '')
    val_pred[i] = val_pred[i].replace(' ', '')

  val_print= {}
  count =0;
  for i in range(val_results.shape[0]):
    val_print[i] = [val_df[0][i],val_df[1][i],val_df[2][i],val_pred[i]]
    if(val_df[0][i]==val_pred[i]):
      count = count+1
      print(val_print[i])
  val_acc = count/val_results.shape[0]
  return val_acc

## Setting up the sweep

In [38]:
def swp():
  #----------------DEFAULT HYPERPARAMETERS--------------------------------------
  hyperparameter_defaults = dict(
      cell_name = "GRU",
      embedding_size = 32,
      hidden_size = 64,
      num_enc_layers = 1,
      num_dec_layers = 1,
      dropout = 0.5,
      r_dropout = 0,
      batch_size = 512,
      num_epochs = 5,
      optimizer = "rmsprop"
  )

  #---------------------RUNNING THE MODEL---------------------------------------
  wandb.init(project="Assignment 3 without Attention", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_cell_{}_embSize_{}_hiddenSize_{}_encLayers_{}_decLayers_{}_dropout_{}_rDropout_{}_batchSize_{}_epochs_{}".format(config.cell_name, 
                    config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, config.num_epochs, 
                    config.optimizer)
  
  enc_dec_model = rnnmodel(config.cell_name, config.embedding_size,config.hidden_size, config.num_enc_layers, config.num_dec_layers, config.dropout, config.r_dropout, config.batch_size, 
         config.num_epochs, config.optimizer)

  #-------------PLOTTING WORD-LEVEL VALIDATION ACCURACY WITH TEACHER FORCING----
  true_val_acc = word_level_acc(enc_dec_model)
  wandb.log({'word_level_val_accuracy':  true_val_acc})

  wandb.run.finish()

In [39]:
sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment03 without Attention",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "cell_name": {
            "values": ['RNN', 'LSTM', 'GRU']
        },
        "embedding_size": {
            "values":[16, 32]
        }, 
        "hidden_size": {
            "values":[64, 128, 256]
        },
        "num_enc_layers":{
            "values":[1, 2, 3]
        },
        "num_dec_layers":{
            "values":[1, 2, 3]
        },  
        "dropout":{
            "values":[0,  0.3,  0.5]
        },
        "r_dropout": {
            "values":[0]
        },
        "batch_size": {
            "values":[32,64,128,256]
        },
        "num_epochs": {
            "values":[10, 15, 20, 25]
        },
        "optimizer": {
            "values":['adam', 'nadam','rmsprop']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: cx0yek50
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/cx0yek50


In [40]:
#wandb.agent('my4rog76', swp)

# BEST MODEL - TEST

In [41]:
#------------CALCULATING WORD-LEVEL TEST ACCURACY BY SAMPLING-------------------
model = tf.keras.models.load_model(wandb.restore('model-best.h5', run_path="kodikarthik21/uncategorized/ey7xk90x").name)

hyperparameter_defaults = dict(
      cell_name = "LSTM",
      embedding_size = 32,
      hidden_size = 128,
      num_enc_layers = 2,
      num_dec_layers = 3,
      dropout = 0.5,
      r_dropout = 0,
      batch_size = 256,
      num_epochs = 25,
      optimizer = "adam"
)

wandb.init(project="Assignment 3 without Attention", config=hyperparameter_defaults)
config = wandb.config

# test_acc = calc_test_acc(model,config)
# print(test_acc)

#BEAM SEARCH

In [42]:

def beam_search(input_seq, k,model,config):
  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  latent_dim = config['hidden_size']
  encoder_inputs = model.input[0]  # input_1
  encoder_seq,*encoder_states  = model.get_layer('encoder_{}'.format(config['num_enc_layers'])).output
  #encoder_seq,state_h_enc, state_c_enc = encoder_lstm2(encoder_seq)  # lstm_
  encoder_model = keras.Model(encoder_inputs, encoder_states)


  decoder_inputs = keras.layers.Input(shape=(1,), name="input_2") # input_2
  decoder_embed = model.get_layer('embedding_2')
  decoder_input = decoder_embed(decoder_inputs)

  if(config['cell_name'] == 'LSTM'):
    decoder_state_input_h = keras.layers.Input(shape=(latent_dim,), name="input_3")
    decoder_state_input_c = keras.layers.Input(shape=(latent_dim,), name="input_4")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  else:
    decoder_states_inputs = keras.layers.Input(shape=(latent_dim,), name="input_3")

  decoder_lstm1 = model.get_layer('decoder_1')
  decoder_outputs, *decoder_states = decoder_lstm1(
      decoder_input, initial_state=decoder_states_inputs
  )

  for i in range(config['num_dec_layers']-1):
    decoder_outputs, *decoder_states = model.get_layer('decoder_'+str(i+2))(
        decoder_outputs,initial_state=decoder_states_inputs
    )

  decoder_dense = model.get_layer('time_distributed')
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs, decoder_states_inputs], [decoder_outputs] + decoder_states
  )
  
#----------------           BEAM SEARCH-----------------------------------------
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)
  # Generate empty target sequence of length 1.
  target_seq = np.zeros((k, 1, 1))
  # Populate the first character of target sequence with the start character.
  target_seq[:, 0, 0] = target_token_index["\t"]

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  sequences = [[list(), 0.0]]
  states = []
  pred_word = [""]
  for x in range(k-1):
    sequences = sequences + [[list(), 0.0]]
    pred_word = pred_word+[""]
  seq_len = 0

  while not stop_condition:
    all_candidates = list()
    
    if seq_len==0:
      seq, score = sequences[0]
      output_tokens, *decoder_states = decoder_model.predict([target_seq[0], states_value])
      states_value = decoder_states
      for j in range(output_tokens.shape[2]):
        candidate = [seq + [j], score - log(output_tokens[0, -1, j])]
        all_candidates.append(candidate)
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      indices = sorted(range(len(all_candidates)), key=lambda x: all_candidates[x][1])[:k]
      sequences = ordered[:k]
      target_seq = np.zeros((len(indices), 1, 1))
      count =0

      for x in range(k):
        states.append(states_value)

      for r in range(len(indices)):
        sampled_char = pred_target_token_index[indices[r]%49]
        target_seq[r, 0, 0] = indices[r]%49
        if sampled_char != "\n" :
            count = 1;  
      if count == 0 or seq_len > max_decoder_seq_length:
        stop_condition = True

    else:
      for l in range(k):
        seq, score = sequences[l]
        output_tokens, *decoder_states = decoder_model.predict([target_seq[l], states[l]])
        states[l] = decoder_states 
        for j in range(output_tokens.shape[2]):
          candidate = [seq + [j], score - log(output_tokens[0, -1, j])]
          all_candidates.append(candidate)
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      indices = sorted(range(len(all_candidates)), key=lambda x: all_candidates[x][1])[:k]
      sequences = ordered[:k]
      target_seq = np.zeros((len(indices), 1, 1))
      count = 0;
      for r in range(len(indices)):
        sampled_char = pred_target_token_index[indices[r]%49]
        target_seq[r, 0, 0] = indices[r]%49
        if sampled_char != "\n" :
            count = 1;
      if count == 0 or seq_len > max_decoder_seq_length:
        stop_condition = True
    seq_len += 1

  for i in range(k):
    #print(sequences[i][0])
    for j in sequences[i][0]:
      if pred_target_token_index[j]!= '\n' and pred_target_token_index[j]!= ' ':
        pred_word[i]  += pred_target_token_index[j]
      else:
        break;
  return pred_word

In [43]:
#-----------------RUNNING BEAM SEARCH ON THE BEST MODEL-------------------------
model = tf.keras.models.load_model(wandb.restore('model-best.h5', run_path="kodikarthik21/uncategorized/ey7xk90x").name)
import random
api = wandb.Api()
run = api.run("kodikarthik21/uncategorized/ey7xk90x")
#------taking random 20 characters and performing beam search on them-----------
for i in range(20):
  index = random.randint(0, test_df.shape[0])
  #s = beam_search(test_encoder_input_data[index:index+1],5,model,run.config)
  #print("English word:" + test_df[1][index] + " True word: " + test_df[0][index] + " Predicted outputs: ", s)

English word:dun True word: டூன் Predicted outputs:  ['டன்', 'டுன்', 'டன்', 'டண்', 'டுண்']


KeyboardInterrupt: ignored